In [ ]:
!pip install psycopg2-binary --user
import os
os._exit(00)

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import numpy as np
import csv, sys, requests
import joblib, psycopg2
import os, json, re

In [2]:
def rs_fetch_datasets():
    user = os.getenv("DKUBE_USER_LOGIN_NAME")
    url = "http://dkube-controller-worker.dkube:5000/dkube/v2/controller/users/%s/datums/class/dataset/datum/%s"
    headers={"authorization": "Bearer "+os.getenv("DKUBE_USER_ACCESS_TOKEN")}
    datasets = []
    for ds in json.load(open('/etc/dkube/redshift.json')):
        resp = requests.get(url % (user, ds.get('rs_name')), headers=headers).json()
        ds['rs_password'] = resp['data']['datum']['redshift']['password']
        datasets.append(ds)
    return datasets

def get_password(user, db):
    datasets = rs_fetch_datasets()
    for dataset in datasets:
        if dataset.get("rs_user") == user and dataset.get("rs_database") == db:
            return dataset.get("rs_password")
    raise Exception("Failed to find password for DB {} with User {}".format(user, db))

In [3]:
def print_records(usr,passwd, hst, prt, db, table):
    try:
        connection = psycopg2.connect(user = usr,
       	                              password = passwd,
           	                          host = hst,
               	                      port = prt,
                   	                  database = db)
    except (Exception, psycopg2.Error) as error :
        print("Error while connecting to Redshift", error)

    cursor = connection.cursor()
    cursor.execute("SELECT * from " + table + ";")
    record = cursor.fetchall()
    for each in record[:10]:
        print(each)

In [4]:
r_endpoint = os.getenv('DKUBE_DATASET_REDSHIFT_ENDPOINT', None)
r_database = os.getenv('DKUBE_DATASET_REDSHIFT_DATABASE', None)
r_user = os.getenv('DKUBE_DATASET_REDSHIFT_USER', None)
r_password = get_password(r_user, r_database)

In [5]:
if 'https:' in r_endpoint:
    p = '(?:https*://)?(?P<host>[^:/ ]+).?(?P<port>[0-9]*).*'
    m = re.search(p,r_endpoint)
else:
    p = '(?:http*://)?(?P<host>[^:/ ]+).?(?P<port>[0-9]*).*'
    m = re.search(p,r_endpoint)

In [6]:
table = 'sklearn'
print_records(r_user, r_password, m.group('host'), m.group('port'), r_database, table)

('26-Feb-16', '708.58', '713.43', '700.86', '705.07', '2239978')
('25-Feb-16', '700.01', '705.98', '690.58', '705.75', '1631855')
('24-Feb-16', '688.92', '700.00', '680.78', '699.56', '1958611')
('23-Feb-16', '701.45', '708.40', '693.58', '695.85', '1999699')
('22-Feb-16', '707.45', '713.24', '702.51', '706.46', '1946067')
('19-Feb-16', '695.03', '703.08', '694.05', '700.91', '1582260')
('18-Feb-16', '710.00', '712.35', '696.03', '697.35', '1859130')
('17-Feb-16', '699.00', '709.75', '691.38', '708.40', '2466808')
('16-Feb-16', '692.98', '698.00', '685.05', '691.00', '2497024')
('12-Feb-16', '690.26', '693.75', '678.60', '682.40', '2129831')
